# Lamat 2024 Winter Bootcamp

Some of this material was adapted from the Lamat 2023 Winter Bootcamp by Jules Fowler and Anne Dattilo.

### Astropy

Astropy is an Python library for all your astrophysical needs! These may include:

- dealing with astronomical coordinate systems
- handling physical quantities and units
- using standard astronomical methods/statistical techniques
- loading and saving files with astronomical data
- and more!

There's a lot you can do with astropy, and I'd recommend checking out their own tutorials at https://learn.astropy.org/ to see what might be relevant for you. I'm going to cover the two aspects of astropy I end up using the most: their units and constants system, and FITS files.

In [ ]:
import astropy
import numpy as np

### Astropy Units and Constants

It's really common for unit issues to give people the wrong answer in scientific computing, because it's often hard to keep track of what units everything's supposed to have in your head, and if you get one wrong it's hard to track it down. To remedy that, astropy provides a set of objects under `astropy.units` representing common units.

In [ ]:
import astropy.units as u
u.m

These are a special type that's specific to astropy:

In [ ]:
type(u.m), type(u.erg)

And there's a lot of them:

In [ ]:
u.s, u.cm, u.K, u.J, u.AU, u.pc

If you want to see what one of them is (it's often hard to parse just off the abbreviation), you can either look up the documentation on the astropy website or access it using `help`:

In [ ]:
help(u.J)

You can turn a number (float) into a `Quantity` object by multiplying a unit:

In [ ]:
d = 1.0 * u.m
d

In [ ]:
type(d)

These units work by only letting you do operations that make sense. For example, you can add together meters and meters:

In [ ]:
1 * u.m + 2 * u.m

Or different units of length:

In [ ]:
1 * u.m + 50 * u.cm

In [ ]:
1000 * u.AU + 0.001 * u.pc

In [ ]:
help(u.pc)

And you can multiply anything:

In [ ]:
2 * u.J + 2 * u.kg * u.m ** 2 / u.s ** 2

In [ ]:
(2 * u.kg * u.m ** 2 / u.s ** 2 + 2 * u.J).to(u.erg)

But you can't add different things together:

In [ ]:
1 * u.m + 2 * u.g

Or do certain operations, like exp and log, on numbers with units:

In [ ]:
np.exp(1 * u.m)

In [ ]:
distance = 3 * u.m
np.log10(distance / u.m)

But if you divide out all units, you can do these:

In [ ]:
np.exp(1 * u.m / (50 * u.cm))

Together with this, we also have some standard constants in `astropy.constants`.

In [ ]:
import astropy.constants as c

In [ ]:
c.h

In [ ]:
c.GM_earth

In [ ]:
help(c)

In [ ]:
(c.h * c.c / (2.82 * c.k_B * 6600 * u.K)).to(u.nm)

**Exercise**: Using the units and constants system, write a function that computes the escape velocity $v = \sqrt{\frac{2GM}{R}}$ of a body with mass $M$ and radius $R$ in units of (divided by) the Earth's escape velocity. Looking at the astropy units documentation at https://docs.astropy.org/en/stable/units/index.html, see if you can find a way to make sure it throws an error if you don't pass in a mass and a radius.

In [ ]:
c.R_earth

In [ ]:
def escape_velocity(M, R):
    # your code here!
    pass

In [ ]:
escape_velocity(c.M_jup, c.R_jup) # this should be about 5.3

In [ ]:
c.R_jup

In [ ]:
escape_velocity(c.R_jup, c.M_jup)

In [ ]:
a = 4
assert a == 3, "I expected a to be 3"

### FITS Files

(credit to Anne Dattilo for most of this section)

A FITS (Flexible Image Transport System) file is a file type that can store multiple types of data. In astronomy, you'll often see it contain image data or data tables (or both!). This is the primary way observational data is stored.

In this tutorial, we will:

- Download data
- Open FITS files and access data
- Edit FITS headers
- Plot FITS images

There are several ways to do each of these things, so feel free to look at the documentation to see other methods!

In [ ]:
from astropy.io import fits
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm, AsinhNorm, PowerNorm
from astropy.visualization import ZScaleInterval

`astropy` has a public data set we will be using to play around with FITS images.

In [ ]:
from astropy.utils.data import download_file

image_file = download_file('http://data.astropy.org/tutorials/FITS-images/HorseHead.fits', cache=True)

In [ ]:
image_file

In [ ]:
type(image_file)

`astropy.fits` has two primary ways of opening FITS files.

The first is `fits.open()`.

This opens the file in a readonly mode and returns an HDU (**H**eader **D**ata **U**nit).

In [ ]:
hdu_list = fits.open(image_file)
hdu_list.info()

`hdu_list.info()` returns an `HDUList` object that gives us information about what's in the FITS file. It acts as a list where we can access each entry like a Python list.

It looks like there's data in the `PRIMARY` extension (`hdu_list[0]`) as well as a Table (`hdu_list[1]`). Let's use the next few cells to inspect the data.

In [ ]:
hdu_list[0].data

In [ ]:
hdu_list[0].data.astype(float) * 3

In [ ]:
# what's the shape of the PrimaryHDU?
hdu_list[0].data.shape

# what is the type of the PrimaryHDU?

In [ ]:
hdu_list.info()

In [ ]:

hdu_list[1].data['ETA']

# what is the type of the TableHDU?

In [ ]:
hdu_list[1].data['ETA'].shape

No matter what you do, if you use `fits.open()` you MUST close your file!!!!!!!!!!!!!

In [ ]:
hdu_list.close()

In [ ]:
hdu_list[0].data

You can access data via HDULists, but I find them slightly confusing. Lucky for us, `astropy.fits` has a direct method of getting your data. This is especially useful if you know what to expect in your file!

In [ ]:
help(fits.getdata)

In [ ]:
image_file

In [ ]:
data = fits.getdata(image_file)

In [ ]:
data

In [ ]:
data, header = fits.getdata(image_file, header=True)

In [ ]:
data

In [ ]:
header["OBJECT"]

In [ ]:
type(header)

The Header Object gives you all the information about the file. In this case, it's giving you observing data for when and where it was taken, as well as much more. This is useful if you need to know the exact time the image was taken, or what filter it was in.

Header objects act like dictionaries, which mean we can ask for different keyword values.

In [ ]:
67.2 * 355 / 3600

In [ ]:
header

In [ ]:
header["OBJECT"]

In [ ]:
header['OBJECT'] = "Horsehead Nebula"

In [ ]:
header["OBJECT"]

In [ ]:
# you can also change the comment at the same time
header["OBSERVER"] = ("Williamina Fleming", "Observation: she did it")

header

In [ ]:
header["OBSERVER"]

We can save these changes:

In [ ]:
fits.writeto('output_file.fits', data, header, overwrite=True)

Finally, let's look at the actual data.

In [ ]:
data

In [ ]:
type(data)

Awesome! It's just a numpy array, which means we can treat it like any other data we use. Let's get to the good stuff and look at it!

In [ ]:
plt.imshow(data)

### Working with Image Data

Last time, we looked at `plt.imshow` in general, but didn't have any way to get data to show. Now that we're comfortable with FITS files, let's see what this image looks like!

Often, it's hard to see astronomical images if you're just plotting pixel intensities, so it's usually good to change the scale or normalization. In `plt.imshow`, we can either set the `vmin` and `vmax` parameters, or the `norm`. In addition to the ones built into matplotlib, `astropy.visualization` has some good methods for this; I like their `ZScaleInterval`, which matches the z-scaling from the plotting tool `ds9`.

In [ ]:
ngc_data = fits.getdata("ngc_b0.fits")

In [ ]:
np.min(ngc_data), np.max(ngc_data)

In [ ]:
vmin, vmax = ZScaleInterval().get_limits(ngc_data)
vmin, vmax

In [ ]:
plt.imshow(ngc_data)
plt.colorbar()
plt.title("NGC 6819 field image")

In [ ]:
# creating fun test data for images
sd_x, sd_y = 5, 5 # try varying these!
grid_size = 10
r = np.arange(-grid_size // 2, grid_size // 2 + grid_size/100, step=grid_size/100)
xg, yg = np.meshgrid(r, r)
z = 1000 / (np.sqrt(2 * np.pi) * sd_x) * np.exp(-xg ** 2 / sd_x ** 2) * 1 / (np.sqrt(2 * np.pi) * sd_y) * np.exp(-yg ** 2 / sd_y ** 2)

In [ ]:
plt.imshow(z, norm=PowerNorm(2)) # I'll vary vmin, vmax, norm here

**Exercise**: Visualize the image in `data`. Give it a colorbar, pick a color map you like, and pick a scaling or normalization that looks good to you.

In [ ]:
# your code here!

**Exercise**: Pick any of the stars in the image and zoom in on it (find approximate coordinates and slice `data` appropriately). Then pick an empty patch of the image and do the same. Using numpy operations (`np.min`, `np.max`, `np.std`, `np.mean`, `np.median`, etc.), come up with a way of describing how much brighter the star is than its background.

In [ ]:
# your code here!